In [3]:
import pandas as pd
import math
import numpy as np
import os
from sklearn import preprocessing
# print(os.getcwd())

In [4]:
# CHANGE the file name of which csv to modify as variable data
# CHANGE working directory as variable wd
# CHANGE variable sample_ratio
wd = "/Users/exequielpunzalan/EECS545/project/bigquery-geotab-intersection-congestion/data/data_to_use/"

# data with weather
data = pd.read_csv(wd + "train_with_weather.csv")
data.rename(columns={'Cityid':'CityId'}, inplace=True)

# # baseline data
# data = pd.read_csv(wd + 'train.csv') # train.csv is from Kaggle
# data['CityId'] = ''

# remove Philadelphia
data = data[data.City!='Philadelphia']
data.loc[data.City=='Atlanta','CityId'] = 3
data.loc[data.City=='Boston','CityId'] = 2
data.loc[data.City=='Chicago','CityId'] = 1

In [6]:
# print(data.shape)
# data.head()
# There are only 235 entries from January and 5 entries from May
data.groupby(['Month']).count()
# dropping January and May entries (don't drop)
# data = data[data.Month != 1]
# data = data[data.Month != 5]

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,...,TimeFromFirstStop_p50,TimeFromFirstStop_p60,TimeFromFirstStop_p80,DistanceToFirstStop_p20,DistanceToFirstStop_p40,DistanceToFirstStop_p50,DistanceToFirstStop_p60,DistanceToFirstStop_p80,City,CityId
Month,,,,,,,,,,,,,,,,,,,,,
1,235,235,235,235,235,235,235,235,235,235,...,235,235,235,235,235,235,235,235,235,235
5,52,52,52,52,52,52,52,52,52,52,...,52,52,52,52,52,52,52,52,52,52
6,63714,63714,63714,63714,63005,63164,63714,63714,63714,63714,...,63714,63714,63714,63714,63714,63714,63714,63714,63714,63714
7,62273,62273,62273,62273,61566,61765,62273,62273,62273,62273,...,62273,62273,62273,62273,62273,62273,62273,62273,62273,62273
8,66834,66834,66834,66834,66058,66241,66834,66834,66834,66834,...,66834,66834,66834,66834,66834,66834,66834,66834,66834,66834
9,64283,64283,64283,64283,63530,63687,64283,64283,64283,64283,...,64283,64283,64283,64283,64283,64283,64283,64283,64283,64283
10,72544,72544,72544,72544,71662,71863,72544,72544,72544,72544,...,72544,72544,72544,72544,72544,72544,72544,72544,72544,72544
11,68770,68770,68770,68770,67934,68176,68770,68770,68770,68770,...,68770,68770,68770,68770,68770,68770,68770,68770,68770,68770
12,70382,70382,70382,70382,69531,69774,70382,70382,70382,70382,...,70382,70382,70382,70382,70382,70382,70382,70382,70382,70382


In [17]:
# sample unique intersections proportionally

sample = pd.DataFrame(columns = data.columns)
# ratio/percentage of samples to get/work with
# e.g. float(10000)/float(468852) = 0.021
sample_ratio = 0.05
np.random.seed(1234)

for i in [1,2,3]:
    IntersectionIds = data.loc[data.CityId==i,'IntersectionId'].unique()
    print(len(IntersectionIds))
    index = np.random.choice(IntersectionIds, size = int(len(IntersectionIds)*sample_ratio), replace=False)
    # in that city and in that intersection
    s = data[(data.CityId==i) & (data.IntersectionId.isin(index))]
    sample = sample.append(s)
    print(sample.shape)

# sample.iloc[:20, :]

2129
(7119, 29)
973
(15745, 29)
377
(20839, 29)


### Encoding

In [18]:
print(sample.shape)
new_sample = sample.copy()

(20839, 29)


In [19]:
# drop the intersection ids whose counts/occurrence are 1
IntersectionIds = data.loc[:,'IntersectionId'].unique()
intersection_id_counts = sample["IntersectionId"].value_counts().to_dict()
drop_ids = [inter_id for inter_id, count in intersection_id_counts.items() if count == 1]
drop_ix = []
for i in drop_ids:
    ix = sample.index[sample['IntersectionId'] == i].tolist()
    drop_ix.append(ix[0]) # because there's only 1 occurrence
new_sample = sample.drop(drop_ix)
print(new_sample.shape)

(20838, 29)


In [20]:
# train, test split data
from sklearn.model_selection import train_test_split
train_sample, test_sample = train_test_split(new_sample, test_size=sample_ratio, stratify = new_sample['IntersectionId'])

In [21]:
# dropping some features and splitting train/test features and labels
train_sample = train_sample.dropna()
X_train = train_sample.drop(['RowId','EntryStreetName','Path',
       'ExitStreetName','TotalTimeStopped_p20', 'TotalTimeStopped_p40',
       'TotalTimeStopped_p50', 'TotalTimeStopped_p60', 'TotalTimeStopped_p80',
       'TimeFromFirstStop_p20', 'TimeFromFirstStop_p40',
       'TimeFromFirstStop_p50', 'TimeFromFirstStop_p60',
       'TimeFromFirstStop_p80', 'DistanceToFirstStop_p20',
       'DistanceToFirstStop_p40', 'DistanceToFirstStop_p50',
       'DistanceToFirstStop_p60', 'DistanceToFirstStop_p80'],axis=1)

labels = ['TotalTimeStopped_p20', 'TotalTimeStopped_p40',
       'TotalTimeStopped_p50', 'TotalTimeStopped_p60', 'TotalTimeStopped_p80',
       'TimeFromFirstStop_p20', 'TimeFromFirstStop_p40',
       'TimeFromFirstStop_p50', 'TimeFromFirstStop_p60',
       'TimeFromFirstStop_p80', 'DistanceToFirstStop_p20',
       'DistanceToFirstStop_p40', 'DistanceToFirstStop_p50',
       'DistanceToFirstStop_p60', 'DistanceToFirstStop_p80']
y_train = train_sample[labels]
# X_train.rename(columns={'Cityid':'CityId'}, inplace=True);

In [22]:
test_sample = test_sample.dropna()
X_test = test_sample.drop(['RowId','EntryStreetName','Path',
       'ExitStreetName','TotalTimeStopped_p20', 'TotalTimeStopped_p40',
       'TotalTimeStopped_p50', 'TotalTimeStopped_p60', 'TotalTimeStopped_p80',
       'TimeFromFirstStop_p20', 'TimeFromFirstStop_p40',
       'TimeFromFirstStop_p50', 'TimeFromFirstStop_p60',
       'TimeFromFirstStop_p80', 'DistanceToFirstStop_p20',
       'DistanceToFirstStop_p40', 'DistanceToFirstStop_p50',
       'DistanceToFirstStop_p60', 'DistanceToFirstStop_p80'],axis=1)
y_test = test_sample[labels]
# X_test.rename(columns={'Cityid':'CityId'}, inplace=True);

In [23]:
# OHE
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# drop City column (don't rerun)
X_train = X_train.drop(['City'], axis=1)
X_test = X_test.drop(['City'], axis=1)
    
# which ones are categorical features
categorical_cols = ['CityId','EntryHeading', 'ExitHeading']
categorical_feature_mask = [True if c in categorical_cols else False for c in X_train.columns]
categorical_feature_mask = pd.Series(data=categorical_feature_mask, index=X_train.columns)

# label encoding
encoder_train = LabelEncoder()
X_train[categorical_cols] = X_train[categorical_cols].apply(encoder_train.fit_transform)
encoder_test = LabelEncoder()
X_test[categorical_cols] = X_test[categorical_cols].apply(encoder_test.fit_transform)


# instantiate OneHotEncoder on categ cols only
# categorical_features = boolean mask for categorical columns
# sparse = False output an array not sparse matrix
# ohe = OneHotEncoder(categorical_features = categorical_feature_mask, sparse=False) 
ohe = OneHotEncoder(sparse=False) 
# apply OneHotEncoder on categorical feature columns
X_ohe_train_ary = ohe.fit_transform(X_train[categorical_cols]) # returns a numpy array
X_ohe_test_ary = ohe.fit_transform(X_test[categorical_cols])

/Applications/anaconda/envs/EECS545-env/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Applications/anaconda/envs/EECS545-env/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, y

In [24]:
# expect 19 additional columns to original data because of OHE, while dropping categorical columns
print(X_train["CityId"].unique()) # 3 unique
print(X_train["EntryHeading"].unique()) # 8 unique
print(X_train["ExitHeading"].unique()) # 8 unique
# shape of dataframes and np array
print(X_train.shape)
print(X_test.shape)
# print(X_train[categorical_cols].shape)
# print(X_test[categorical_cols].shape)
print(X_ohe_train_ary.shape)
print(X_ohe_test_ary.shape)

[1 2 0]
[5 0 1 7 6 4 3 2]
[5 4 7 0 3 6 2 1]
(19355, 9)
(1020, 9)
(19355, 19)
(1020, 19)


In [25]:
# append OHE vectors to dataframes
num_unique_CityId = len(X_train["CityId"].unique())
num_unique_EntryHeading = len(X_train["EntryHeading"].unique())
num_unique_ExitHeading = len(X_train["ExitHeading"].unique())

for i in range(num_unique_CityId):
    # add new column where the column has 0/1 values for that feature
    X_train[categorical_cols[0] + str(i)] = X_ohe_train_ary[:, i]
    X_test[categorical_cols[0] + str(i)] = X_ohe_test_ary[:, i]
for i in range(num_unique_EntryHeading):
    X_train[categorical_cols[1] + str(i)] = X_ohe_train_ary[:, i+num_unique_CityId]
    X_test[categorical_cols[1] + str(i)] = X_ohe_test_ary[:, i+num_unique_CityId]
for i in range(num_unique_ExitHeading):
    X_train[categorical_cols[2] + str(i)] = X_ohe_train_ary[:, i+num_unique_CityId+num_unique_EntryHeading]
    X_test[categorical_cols[2] + str(i)] = X_ohe_test_ary[:, i+num_unique_CityId+num_unique_EntryHeading]

In [26]:
# sort column names and save to csv
X_train = X_train.reindex(sorted(X_train.columns), axis=1)
X_train = X_train.drop(categorical_cols, axis=1)
X_test = X_test.reindex(sorted(X_test.columns), axis=1)
X_test = X_test.drop(categorical_cols, axis=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
X_train.head(10)
# saving
X_train.to_csv(wd + "formatted_train_data_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)
X_test.to_csv(wd + "formatted_test_data_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)
y_train.to_csv(wd + "formatted_train_labels_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)
y_test.to_csv(wd + "formatted_test_labels_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)

# X_train.to_csv(wd + "baseline_train_data_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)
# X_test.to_csv(wd + "baseline_test_data_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)
# y_train.to_csv(wd + "baseline_train_labels_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)
# y_test.to_csv(wd + "baseline_test_labels_sampleratio{}.csv".format(sample_ratio), encoding='utf-8', index=False)

(19355, 25)
(1020, 25)
(19355, 15)
(1020, 15)
